Notes: Valves and similar components belong to the PipingNetworkSystem domain; they do not represent an independent category. Moreover, they are subordinated to a PipingNetworkSegment.<br>
More specifically to the one, that starts from this Component.

In [4]:
import Path
from FAIRFlowChemistry.tools.reader import DEXPI2sdRDM

In [5]:
cwd = Path.cwd()
filepath = cwd / "data" / "DEXPI" / "DEXPI_PID.xml" #"P&ID.xml"
plant_setup = DEXPI2sdRDM( filepath)

GasCylinder-1 ['Valve-1']
GasCylinder-2 ['Valve-1']
Vessel-1 ['Valve-1', 'Valve-2']
GasFilter-1 ['Valve-2']
Valve-1 ['GasCylinder-1', 'Vessel-1', 'GasCylinder-2']
Valve-2 ['Vessel-1', 'GasFilter-1']


PlantSetup(id='374f72ed-7f50-4346-8714-dfbef6b16521', components=[Component(id='9a9f0164-b31f-4ed5-8ae1-4f87f8aa7390', component_type='Equipment', component_id='GasCylinder-1', component_class='GasCylinder', component_class_uri='https://data.posccaesar.org/rdl/RDS4316755815', component_name=None, generic_attributes=[], connections=['Valve-1']), Component(id='1e9dc67e-a224-451a-977f-22406806b45d', component_type='Equipment', component_id='GasCylinder-2', component_class='GasCylinder', component_class_uri='https://data.posccaesar.org/rdl/RDS4316755815', component_name=None, generic_attributes=[], connections=['Valve-1']), Component(id='45363515-8d79-40a0-9f45-f13a6d37f44b', component_type='Equipment', component_id='Vessel-1', component_class='Vessel', component_class_uri='https://data.posccaesar.org/rdl/RDS414674', component_name=None, generic_attributes=[], connections=['Valve-1', 'Valve-2']), Component(id='27729983-d2ab-40fe-922b-1e64166008ac', component_type='Equipment', component_id=

In [8]:
plant_setup.components[0]

Component(id='f9a0e523-0947-4f1e-bfa0-aca2aa24d6be', component_type='Equipment', component_id='GasCylinder-1', component_class='GasCylinder', component_class_uri='https://data.posccaesar.org/rdl/RDS4316755815', component_name=None, generic_attributes=[], connections=['Valve-1'])

In [ ]:
###  GenericAttibute

- name
  - Type: string
  - Description: bla.
- attribute_uri
  - Type: string
  - Description: bla.
- value
  - Type: string
  - Description: bla.
- format
  - Type: string
  - Description: bla.
- units
  - Type: string
  - Description: bla.
- units_uri
  - Type: string
  - Description: bla